# A nanoGPT Decoder

Implementing a self-attention and feed forward architecture for GPT decoding. 

It mimics GPT with its scaled behavior by tockenizing and generating shakespeare.

Base skeleton code is from: https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing

# Data Processing

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/shakespeare/text.txt


In [2]:
text = open('../input/shakespeare/text.txt').read() 
print('Length:', len(text))
# Print first 60 characters
print(text[:60])

Length: 1115393
First Citizen:
Before we proceed any further, hear me speak.


**Building our vocabulary and Tokenization Logic**

In [3]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Vocab Size: ", vocab_size)
print(chars)

Vocab Size:  65
['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


Character tokenization. Google has **SentencePiece** sub-word units tokenizer while OpenAI uses **tiktoken**. These are not character level but not word level either. Rather, they are short sub-word length tokens.

In [4]:
# String to Integer
stoi = {ch:i for i,ch in enumerate(chars)}
# Integer to String
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s] 
decode = lambda l: ''.join([itos[i] for i in l])

print(encode("hello world"))
print(decode(encode("hello world")))

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


**Tockenize entire chunk of shakespeare.**

In [5]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
# Print first 60 encoded characters
print(data.shape, data.dtype)
print(data[:60])

torch.Size([1115393]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8])


Separate dataset into train and validation (splitting) at a splitting percentage.

In [6]:
portion = 0.9
n = int(portion*len(data))
train_data = data[:n]
val_data = data[n:]

# Data Loading
Creating batches of our data to feed our eventual transformer, because putting them all at once is simply too computationally taxing. The transformer will be able to predict sequences from 1 to the maximal block_size. For inputs longer than block_size they are truncated.

In [7]:
block_size = 8 # maximum context length for predictions
batch_size = 4 # how many sequences we are parallel processing at once

In [8]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y # x is the training and y is the answers

xb, yb = get_batch('train')
print("Inputs:")
print(xb.shape)
print(xb)
print("Targets:")
print(yb.shape)
print(yb)



Inputs:
torch.Size([4, 8])
tensor([[63,  1, 40, 43, 45, 47, 52, 57],
        [ 0,  0, 16, 33, 23, 17,  1, 34],
        [56, 53, 61, 11,  0, 19, 50, 53],
        [43, 52, 58,  0, 32, 46, 43,  1]])
Targets:
torch.Size([4, 8])
tensor([[ 1, 40, 43, 45, 47, 52, 57,  1],
        [ 0, 16, 33, 23, 17,  1, 34, 21],
        [53, 61, 11,  0, 19, 50, 53, 60],
        [52, 58,  0, 32, 46, 43,  1, 59]])


# Modeling (BigramLanguageModel)

Very simple model as a baseline.

In [9]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8089, grad_fn=<NllLossBackward0>)


**Testing without training:**
Expect Jibberish

In [10]:
pred = model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)
print(decode(pred[0].tolist()))


fWBEuvY'YR;kSVN.ID  qVD C3:r'.,e:blFWDlW.rTXjpeBNYhcomdRvNOwadiySTx'&PqExJaZcXOZ&G
sm 'HMtkXnpfcJyvK


Training this baseline model over 15000 iterations/epochs to get a closer generation (but still not close at all).

In [11]:
# creating the optimizer object.
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

batch_size = 32
epochs = 15000
for steps in range(epochs):
    
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.569352865219116


In [12]:
idx = torch.zeros((1,1), dtype=torch.long)
pred = model.generate(idx, max_new_tokens=100)
print(decode(pred[0].tolist()))


o dsman nghe:
F thite nd
Yls will!
KIO -merehen y tot theneee RWeff he resis ve lil anoilery w INAy 


# Loss Improvements
An improved loss estimation function to remove noise. And Hyperparameter tuning.

In [13]:
batch_size = 16 
block_size = 32 
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
eval_iters = 200

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 2.5564, val loss 2.5619
step 100: train loss 2.5510, val loss 2.5548
step 200: train loss 2.5358, val loss 2.5435
step 300: train loss 2.5351, val loss 2.5372
step 400: train loss 2.5226, val loss 2.5343
step 500: train loss 2.5202, val loss 2.5325
step 600: train loss 2.5208, val loss 2.5187
step 700: train loss 2.5120, val loss 2.5218
step 800: train loss 2.5079, val loss 2.5151
step 900: train loss 2.5045, val loss 2.5156
step 1000: train loss 2.5001, val loss 2.5217
step 1100: train loss 2.4987, val loss 2.5094
step 1200: train loss 2.4983, val loss 2.5057
step 1300: train loss 2.4950, val loss 2.5020
step 1400: train loss 2.4864, val loss 2.5079
step 1500: train loss 2.4952, val loss 2.5000
step 1600: train loss 2.4894, val loss 2.5025
step 1700: train loss 2.4894, val loss 2.4957
step 1800: train loss 2.4795, val loss 2.4938
step 1900: train loss 2.4858, val loss 2.4941
step 2000: train loss 2.4815, val loss 2.4922
step 2100: train loss 2.4829, val loss 2.4943


# Mathematical Improvements
Mathematical trick for self-attention block.
Allowing tokens to communicate 

In [14]:
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [15]:
# version 1
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0)

In [16]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

True

In [17]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)

True

You can do weighted aggregations of your past elements by matrix multiplying a lower triangular matrix with entires that correspond to how much of each past element should fuse.

An approach that follows this basic concept is called **self-attention**.

In [18]:
# version 4: self-attention!
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.

In [19]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [20]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [21]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-2.0862e-08), tensor(1.0000))

# Remaking Model

In [22]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

text = open('../input/shakespeare/text.txt').read() 

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.2826, val loss 4.2839
step 100: train loss 2.6554, val loss 2.6635
step 200: train loss 2.5098, val loss 2.5090
step 300: train loss 2.4190, val loss 2.4304
step 400: train loss 2.3530, val loss 2.3676
step 500: train loss 2.3048, val loss 2.3038
step 600: train loss 2.2451, val loss 2.2536
step 700: train loss 2.2090, val loss 2.2159
step 800: train loss 2.1656, val loss 2.1882
step 900: train loss 2.1274, val loss 2.1757
step 1000: train loss 2.1018, val loss 2.1339
step 1100: train loss 2.0549, val loss 2.1146
step 1200: train loss 2.0526, val loss 2.1136
step 1300: train loss 2.0144, val loss 2.0777
step 1400: train loss 1.9784, val loss 2.0578
step 1500: train loss 1.9711, val loss 2.0321
step 1600: train loss 1.9570, val loss 2.0390
step 1700: train loss 1.9261, val loss 2.0073
step 1800: train loss 1.9134, val loss 1.9969
step 1900: train loss 1.8831, val loss 1.9795
step 2000: train loss 1.8766, val loss 1.9697
step 2100: train loss 1.